In [12]:
import pandas as pd
import sqlite3
import numpy as np

conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""SELECT
                        team,
                        is_home,
                        match_date,
                        division,
                        season,
                        match_url,
                        summary_goals,
                        summary_pens_made,
                        summary_shots,
                        summary_shots_on_target,
                        summary_touches,
                        summary_xg,
                        summary_npxg,
                        possession_touches_att_pen_area,
                        keeper_psxg,
                        summary_cards_red,
                        opp_team,
                        opp_summary_goals,
                        opp_summary_pens_made,
                        opp_summary_shots,
                        opp_summary_shots_on_target,
                        opp_summary_touches,
                        opp_summary_xg,
                        opp_summary_npxg,
                        opp_possession_touches_att_pen_area,
                        opp_keeper_psxg,
                        opp_summary_cards_red            
                       FROM fbref_match_all_columns
                       WHERE division IN ('Premier League', 'Championship')""", conn)

df

,team,is_home,match_date,division,season,match_url,summary_goals,summary_pens_made,summary_shots,summary_shots_on_target,...,opp_summary_goals,opp_summary_pens_made,opp_summary_shots,opp_summary_shots_on_target,opp_summary_touches,opp_summary_xg,opp_summary_npxg,opp_possession_touches_att_pen_area,opp_keeper_psxg,opp_summary_cards_red
0,Tottenham,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/e4bb1c35/Tottenha...,1.0,1.0,3.0,1.0,...,4.0,1.0,22.0,7.0,691.0,2.2,1.5,44.0,4.1,0.0
1,Brighton,0,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/e4bb1c35/Tottenha...,4.0,1.0,22.0,7.0,...,1.0,1.0,3.0,1.0,424.0,2.0,1.2,10.0,1.5,0.0
2,Bournemouth,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/1ff370e8/Bournemo...,2.0,0.0,20.0,7.0,...,0.0,0.0,3.0,0.0,439.0,0.3,0.3,12.0,0.0,0.0
3,Leicester City,0,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/1ff370e8/Bournemo...,0.0,0.0,3.0,0.0,...,2.0,0.0,20.0,7.0,647.0,1.6,1.6,48.0,2.1,0.0
4,Newcastle Utd,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/36844e73/Newcastl...,0.0,0.0,17.0,6.0,...,1.0,0.0,14.0,6.0,419.0,1.2,1.2,13.0,1.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13867,Leeds United,0,2020-09-12,Premier League,2020-2021,https://fbref.com/en/matches/21b58926/Liverpoo...,3.0,0.0,6.0,3.0,...,4.0,2.0,20.0,4.0,612.0,2.7,1.2,44.0,2.5,0.0
13868,Crystal Palace,1,2020-09-12,Premier League,2020-2021,https://fbref.com/en/matches/db261cb0/Crystal-...,1.0,0.0,5.0,3.0,...,0.0,0.0,9.0,5.0,746.0,0.9,0.9,21.0,0.4,0.0
13869,Southampton,0,2020-09-12,Premier League,2020-2021,https://fbref.com/en/matches/db261cb0/Crystal-...,0.0,0.0,9.0,5.0,...,1.0,0.0,5.0,3.0,400.0,1.1,1.1,14.0,0.8,0.0
13870,Fulham,1,2020-09-12,Premier League,2020-2021,https://fbref.com/en/matches/bf52349b/Fulham-A...,0.0,0.0,5.0,2.0,...,3.0,0.0,13.0,5.0,742.0,2.6,2.6,26.0,1.8,0.0


In [13]:
promoted_teams = pd.read_csv("promoted_teams.csv")

promoted_teams

,team,division,season
0,Newcastle Utd,Premier League,2017-2018
1,Brighton,Premier League,2017-2018
2,Huddersfield,Premier League,2017-2018
3,Wolves,Premier League,2018-2019
4,Cardiff City,Premier League,2018-2019
...,...,...,...
141,Valladolid,La Liga,2022-2023
142,Girona,La Liga,2022-2023
143,Leganés,La Liga,2024-2025
144,Valladolid,La Liga,2024-2025


In [15]:
import pandas as pd
import numpy as np
from numba import jit

@jit(nopython=True)
def calculate_weights_vectorized(days_diff, red_cards, decay_rate=0.0077):
    """Vectorized weight calculation using numba for speed"""
    weights = np.exp(-days_diff * decay_rate)
    # Reduce weight for matches with red cards
    weights = np.where(red_cards == 1, weights * 0.3, weights)
    return weights

def apply_weighted_avg_vectorized(values, days_diff, red_cards, 
                                decay_rate=0.0077, time_window=180, 
                                min_games=5, recent_game_window=120):
    """Optimized weighted average calculation"""
    # Create masks once
    valid_mask = ~pd.isna(values)
    time_window_mask = days_diff <= time_window
    recent_mask = days_diff <= recent_game_window
    
    # Combined mask
    combined_mask = valid_mask & time_window_mask
    
    # Early returns
    if not combined_mask.any():
        return np.nan
    if combined_mask.sum() < min_games:
        return np.nan
    if not (valid_mask & recent_mask).any():
        return np.nan
    
    # Filter data once
    valid_values = values[combined_mask]
    valid_days = days_diff[combined_mask]
    valid_red = red_cards[combined_mask]
    
    # Calculate weights
    weights = calculate_weights_vectorized(valid_days, valid_red, decay_rate)
    
    # Calculate weighted average
    try:
        valid_values_numeric = pd.to_numeric(valid_values, errors='coerce')
        if pd.isna(valid_values_numeric).all():
            return np.nan
        
        # Remove NaN values from conversion
        final_mask = ~pd.isna(valid_values_numeric)
        if not final_mask.any():
            return np.nan
            
        final_values = valid_values_numeric[final_mask]
        final_weights = weights[final_mask]
        
        return np.sum(final_weights * final_values) / np.sum(final_weights)
    except:
        return np.nan

def process_team_metrics_optimized(df):
    """Optimized processing of team metrics"""
    
    # Clean and prepare data (same as original)
    df = df.drop_duplicates(subset=['match_url', 'team'])
    df['match_date'] = pd.to_datetime(df['match_date'])
    df['match_red'] = df["summary_cards_red"].astype(int) + df["opp_summary_cards_red"].astype(int)
    df['match_id'] = df['match_url']

    # Identify numeric columns (same logic as original)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    exclude_cols = ['match_red', 'summary_cards_red', 'opp_summary_cards_red', 'is_home']
    numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

    team_stats = [col for col in numeric_cols if not col.startswith('opp_')]
    opp_stats = [col for col in numeric_cols if col.startswith('opp_')]

    print(f"Processing {len(team_stats)} team stats and {len(opp_stats)} opponent stats")

    # Sort data once for all teams
    df_sorted = df.sort_values(['team', 'match_date']).reset_index(drop=True)
    
    # Pre-calculate all stats columns we'll need
    all_stats = team_stats + opp_stats
    
    results = []
    
    # Group by team to process each team's matches
    for team_name, team_data in df_sorted.groupby('team'):
        team_data = team_data.reset_index(drop=True)
        n_matches = len(team_data)
        
        if n_matches <= 1:
            continue
            
        # Pre-compute match dates as numpy array for faster operations
        match_dates = team_data['match_date'].values
        match_red = team_data['match_red'].values
        
        # Process each match (starting from index 1, same as original)
        for current_idx in range(1, n_matches):
            current_match = team_data.iloc[current_idx]
            current_date = match_dates[current_idx]
            
            # Calculate days difference for all previous matches at once
            prev_dates = match_dates[:current_idx]
            prev_red = match_red[:current_idx]
            days_diff = (current_date - prev_dates) / np.timedelta64(1, 'D')
            
            metrics_dict = {
                'team': team_name,
                'match_id': current_match['match_id'],
                'opp_team': current_match['opp_team']
            }
            
            # Process team stats
            for stat in team_stats:
                prev_values = team_data[stat].iloc[:current_idx]
                
                weighted_avg = apply_weighted_avg_vectorized(
                    prev_values.values, days_diff, prev_red
                )
                metrics_dict[f'team_rolling_{stat}'] = weighted_avg
            
            # Process opponent stats (defensive perspective)
            for stat in opp_stats:
                prev_values = team_data[stat].iloc[:current_idx]
                
                weighted_avg = apply_weighted_avg_vectorized(
                    prev_values.values, days_diff, prev_red
                )
                
                clean_stat = stat.replace('opp_', '')
                metrics_dict[f'team_rolling_conceded_{clean_stat}'] = weighted_avg
            
            results.append(metrics_dict)
    
    # Convert to DataFrame once
    metrics_df = pd.DataFrame(results)
    
    # Merge operations (same as original but more efficient)
    final_df = df.merge(
        metrics_df,
        on=['team', 'match_id', 'opp_team'],
        how='left'
    )

    # Create opponent metrics
    opp_metrics_df = metrics_df.copy()
    opp_metrics_df = opp_metrics_df.rename(columns={
        'team': 'opp_team',
        'opp_team': 'team'
    })

    rolling_cols = [col for col in opp_metrics_df.columns if col.startswith('team_rolling_')]
    rename_dict = {col: f'opp_{col}' for col in rolling_cols}
    opp_metrics_df = opp_metrics_df.rename(columns=rename_dict)

    final_df = final_df.merge(
        opp_metrics_df,
        on=['team', 'match_id', 'opp_team'],
        how='left'
    )

    # Select final columns (same as original)
    key_cols = ['match_url', 'match_date', 'season', 'division', 'team', 'opp_team', 
                'is_home']
    rolling_cols = [col for col in final_df.columns if 'rolling' in col]
    feature_cols = key_cols + rolling_cols
    features_df = final_df[feature_cols].copy()
    
    return features_df

# Usage (same interface as original)
features_df = process_team_metrics_optimized(df)

Processing 9 team stats and 9 opponent stats


In [16]:
features_df['gw'] = features_df.groupby(['team', 'season', 'division'])['match_date'].rank(method='dense').astype(int)
features_df['is_early_season?'] = np.where(features_df['gw'] < 10, 1, 0)
features_df['is_promoted?'] = features_df.merge(promoted_teams, on=['team', 'division', 'season'], how='left', indicator=True)['_merge'].eq('both').astype(int)

features_df

,match_url,match_date,season,division,team,opp_team,is_home,team_rolling_summary_goals,team_rolling_summary_pens_made,team_rolling_summary_shots,...,opp_team_rolling_conceded_summary_shots,opp_team_rolling_conceded_summary_shots_on_target,opp_team_rolling_conceded_summary_touches,opp_team_rolling_conceded_summary_xg,opp_team_rolling_conceded_summary_npxg,opp_team_rolling_conceded_possession_touches_att_pen_area,opp_team_rolling_conceded_keeper_psxg,gw,is_early_season?,is_promoted?
0,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25,2024-2025,Premier League,Tottenham,Brighton,1,1.289346,0.099220,11.135216,...,10.857144,3.427652,593.691838,1.339005,1.150561,23.135899,1.269035,38,0,0
1,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25,2024-2025,Premier League,Brighton,Tottenham,0,1.756276,0.301971,13.371462,...,14.976592,5.211294,577.666107,1.803889,1.770927,27.321844,1.890708,38,0,0
2,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25,2024-2025,Premier League,Bournemouth,Leicester City,1,1.620226,0.133736,14.451036,...,16.524494,4.915440,671.768114,1.915188,1.787126,29.916176,1.738255,38,0,0
3,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25,2024-2025,Premier League,Leicester City,Bournemouth,0,0.767395,0.000000,8.679818,...,12.755894,4.204451,593.777537,1.319921,1.188969,24.624059,1.354963,38,0,1
4,https://fbref.com/en/matches/36844e73/Newcastl...,2025-05-25,2024-2025,Premier League,Newcastle Utd,Everton,1,2.071963,0.170780,13.238090,...,12.095816,4.350498,669.491722,1.138803,1.112165,23.319336,1.247472,38,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13867,https://fbref.com/en/matches/21b58926/Liverpoo...,2020-09-12,2020-2021,Premier League,Leeds United,Liverpool,0,2.334007,0.120432,16.862586,...,7.383654,2.586850,493.308949,1.196898,1.112476,10.199228,1.124304,1,1,1
13868,https://fbref.com/en/matches/db261cb0/Crystal-...,2020-09-12,2020-2021,Premier League,Crystal Palace,Southampton,1,0.588576,0.000000,10.089321,...,10.857775,2.994314,554.864046,1.218138,1.218138,20.902557,1.221304,1,1,0
13869,https://fbref.com/en/matches/db261cb0/Crystal-...,2020-09-12,2020-2021,Premier League,Southampton,Crystal Palace,0,1.922152,0.136975,13.973167,...,12.932078,4.419749,690.958959,1.499584,1.499584,21.356376,1.824323,1,1,0
13870,https://fbref.com/en/matches/bf52349b/Fulham-A...,2020-09-12,2020-2021,Premier League,Fulham,Arsenal,1,1.309914,0.120436,13.410356,...,13.505428,4.917995,576.533198,1.614600,1.476587,23.536873,1.376875,1,1,1


In [ ]:
features_df.to_sql('prem_team_goal_features', conn, if_exists='replace', index=False)

conn.close()